In [3]:
#Importando Bibliotecas Pandas e Numpy
import pandas as pd
import numpy as np
import os, sys
import time

#Importando BiBliotecas DASK  (alternativa ao Pandas)
import dask.dataframe as dd

#Otimizador Giovanni
from sklearnex import patch_sklearn 
patch_sklearn()


CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional/TP1_csvs" # Path Giovanni
# CSV_PATH = "E:/Documentos/CEFET/OneDrive/Documentos/2ECOM067_INTELIGENCIA-COMPUTACIONAL-I_T01/TP1/bases/" # Path Jorge

import warnings
warnings.simplefilter("ignore")
#Carregando Base de dados principal
df_test  = pd.read_csv(f"{CSV_PATH}/application_test.csv")
df_train = pd.read_csv(f"{CSV_PATH}/application_train.csv")

# Fazendo o merge com os dados auxiliares
df_auxiliar_bureau                          = pd.read_csv(f"{CSV_PATH}/bureau.csv")
df_auxiliar_bureau_balance                  = pd.read_csv(f"{CSV_PATH}/bureau_balance.csv")
df_auxiliar_previous_application            = pd.read_csv(f"{CSV_PATH}/previous_application.csv")
df_auxiliar_POS_CASH_balance                = pd.read_csv(f"{CSV_PATH}/POS_CASH_balance.csv")
df_auxiliar_installments_payments           = pd.read_csv(f"{CSV_PATH}/installments_payments.csv")
df_auxiliar_credit_card_balances            = pd.read_csv(f"{CSV_PATH}/credit_card_balance.csv")


#df_train = df_train.sample(frac=0.10, random_state=42)  # Seleciona 10% do DataFrame de treino
df_train = df_train.join(df_auxiliar_bureau, on='SK_ID_CURR', lsuffix = '_duplicate', how='left')
df_train = df_train.join(df_auxiliar_bureau_balance, on='SK_ID_BUREAU', lsuffix = '_duplicate2', how='left')
df_train = df_train.join(df_auxiliar_previous_application, on='SK_ID_CURR', lsuffix = '_duplicate3', how='left')
df_train = df_train.join(df_auxiliar_POS_CASH_balance, on='SK_ID_PREV', lsuffix = '_duplicate4', how='left')
df_train = df_train.join(df_auxiliar_installments_payments, on='SK_ID_PREV', lsuffix = '_duplicate5', how='left')
df_train = df_train.join(df_auxiliar_credit_card_balances, on='SK_ID_PREV', lsuffix = '_duplicate6', how='left')

#Dropando colunas duplicadas
sufixos = ['_duplicate', '_duplicate2', '_duplicate3', '_duplicate4', '_duplicate5', '_duplicate6']
df_train = df_train.loc[:, ~df_train.columns.str.endswith(tuple(sufixos))]


# Removendo caracteres especiais das colunas
import re
df_train.columns = [re.sub(r'[^\w]', '', col) for col in df_train.columns]

# Salvando o DataFrame resultante em um arquivo CSV
df_train.to_csv(f"df_train_full.csv", index=False)
print(df_train.shape)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


(307511, 195)


In [4]:
df_train.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 195 columns):
 #    Column                        Non-Null Count   Dtype  
---   ------                        --------------   -----  
 0    TARGET                        307511 non-null  int64  
 1    CODE_GENDER                   307511 non-null  object 
 2    FLAG_OWN_CAR                  307511 non-null  object 
 3    FLAG_OWN_REALTY               307511 non-null  object 
 4    CNT_CHILDREN                  307511 non-null  int64  
 5    AMT_INCOME_TOTAL              307511 non-null  float64
 6    NAME_INCOME_TYPE              307511 non-null  object 
 7    NAME_EDUCATION_TYPE           307511 non-null  object 
 8    NAME_FAMILY_STATUS            307511 non-null  object 
 9    NAME_HOUSING_TYPE             307511 non-null  object 
 10   REGION_POPULATION_RELATIVE    307511 non-null  float64
 11   DAYS_BIRTH                    307511 non-null  int64  
 12   DAYS_EMPLOYED               

In [6]:
# Criando csv completo de teste
df_test = df_test.join(df_auxiliar_bureau, on='SK_ID_CURR', lsuffix = '_duplicate', how='left')
df_test = df_test.join(df_auxiliar_bureau_balance, on='SK_ID_BUREAU', lsuffix = '_duplicate2', how='left')
df_test = df_test.join(df_auxiliar_previous_application, on='SK_ID_CURR', lsuffix = '_duplicate3', how='left')
df_test = df_test.join(df_auxiliar_POS_CASH_balance, on='SK_ID_PREV', lsuffix = '_duplicate4', how='left')
df_test = df_test.join(df_auxiliar_installments_payments, on='SK_ID_PREV', lsuffix = '_duplicate5', how='left')
df_test = df_test.join(df_auxiliar_credit_card_balances, on='SK_ID_PREV', lsuffix = '_duplicate6', how='left')

#Dropando colunas duplicadas
sufixos = ['_duplicate', '_duplicate2', '_duplicate3', '_duplicate4', '_duplicate5', '_duplicate6']
df_test = df_test.loc[:, ~df_test.columns.str.endswith(tuple(sufixos))]

# Removendo caracteres especiais das colunas
df_test.columns = [re.sub(r'[^\w]', '', col) for col in df_test.columns]

# Criando csv completo de teste
df_test.to_csv(f"df_test.csv", index=False)
print(df_test.shape)    

(48744, 194)


In [7]:
df_test.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 194 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    CODE_GENDER                   48744 non-null  object 
 1    FLAG_OWN_CAR                  48744 non-null  object 
 2    FLAG_OWN_REALTY               48744 non-null  object 
 3    CNT_CHILDREN                  48744 non-null  int64  
 4    AMT_INCOME_TOTAL              48744 non-null  float64
 5    NAME_INCOME_TYPE              48744 non-null  object 
 6    NAME_EDUCATION_TYPE           48744 non-null  object 
 7    NAME_FAMILY_STATUS            48744 non-null  object 
 8    NAME_HOUSING_TYPE             48744 non-null  object 
 9    REGION_POPULATION_RELATIVE    48744 non-null  float64
 10   DAYS_BIRTH                    48744 non-null  int64  
 11   DAYS_EMPLOYED                 48744 non-null  int64  
 12   DAYS_REGISTRATION             48744 non-null

In [5]:
# Aliviando memoria
del df_auxiliar_bureau, df_auxiliar_bureau_balance, df_auxiliar_previous_application 
del df_auxiliar_POS_CASH_balance, df_auxiliar_installments_payments, df_auxiliar_credit_card_balances

In [14]:
# Verificando as colunas de treino
with open("attributes_train_100.txt", 'w') as file:
    for col in df_train.columns:
        file.write(col + "\n")

In [ ]:
# Verificando as colunas de teste
with open("attributes_test.txt", 'w') as file:
    for col in df_test.columns:
        file.write(col + "\n")
